In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
from datetime import datetime

/Users/ardadinc/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


<h3>-----------------------------Stock_Anaylsis.com Parsing Code-----------------------------<h3>
<h6>url = 'https://stockanalysis.com/actions/splits/'<h6>


In [2]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd

# URL of the webpage that contains stock split data
url = 'https://stockanalysis.com/actions/splits/'

# Custom headers to mimic a browser request
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# Making the GET request
response = requests.get(url, headers=headers)

# Check if the request was successful
if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')
    # Find the table or div that contains the stock split data
    table = soup.find('table')  # Assuming the data is in a table
    if table:
        rows = table.find_all('tr')
        data = []
        for row in rows[1:]:  # Skipping the header row
            cells = row.find_all('td')
            if len(cells) > 1:
                date_str = cells[0].text.strip()
                date = datetime.strptime(date_str, "%b %d, %Y").strftime("%-m/%-d/%Y")
                symbol = cells[1].text.strip()
                company_name = cells[2].text.strip()
                split_type = cells[3].text.strip()
                split_ratio = cells[4].text.strip().replace(" for ", " : ")
                if "forward" not in split_type.lower():
                    data.append([date, symbol, company_name, split_ratio])
        
        # Creating DataFrame without the Type column
        df = pd.DataFrame(data, columns=['Date', 'Symbol', 'Company Name', 'Split Ratio'])
        
        # Saving DataFrame to CSV
        output_file_path = '/Users/ardadinc/Desktop/Market-Insight/Web_Parse_Data/Parsed_Data/StockAnalysis_Split_Data.csv'
        df.to_csv(output_file_path, index=False)
        print(f"Data has been saved to {output_file_path}")
    else:
        print("Failed to find the table containing stock split data.")
else:
    print(f"Failed to retrieve data. Status code: {response.status_code}")


Data has been saved to /Users/ardadinc/Desktop/Market-Insight/Web_Parse_Data/Parsed_Data/StockAnalysis_Split_Data.csv


<h3>-----------------------------TipRanks.com Parsing Code-----------------------------<h3>
<h6>url = 'https://www.tipranks.com/calendars/stock-splits/upcoming'<h6>


In [3]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd

# Function to extract and process stock split data from tipranks.com
def get_tipranks_data():
    url = 'https://www.tipranks.com/calendars/stock-splits/upcoming'
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    response = requests.get(url, headers=headers)
    data = []
    
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        table = soup.find('table')
        if table:
            rows = table.find_all('tr')
            for row in rows[1:]:  # Skip header row
                cells = row.find_all('td')
                if len(cells) > 1:
                    split_date = cells[0].text.strip()
                    split_date = datetime.strptime(split_date, "%b %d, %Y").strftime("%-m/%-d/%Y")
                    symbol = cells[1].text.strip()
                    company_name = cells[2].text.strip()
                    split_type = cells[3].text.strip()
                    split_ratio = cells[4].text.strip().replace(" for ", " : ")
                    if "forward" not in split_type:
                        data.append([split_date, symbol, company_name, split_ratio])
    else:
        print(f"Failed to retrieve data. Status code: {response.status_code}")
    
    return data

# Extract data from tipranks.com
tipranks_data = get_tipranks_data()

# Create DataFrame and save to CSV without the Type column
df_tipranks = pd.DataFrame(tipranks_data, columns=['Date', 'Symbol', 'Company Name', 'Split Ratio'])
output_file_path = '/Users/ardadinc/Desktop/Market-Insight/Web_Parse_Data/Parsed_Data/Tipranks_Split_Data.csv'
df_tipranks.to_csv(output_file_path, index=False)
print(f"Tipranks data has been saved to {output_file_path}")


Tipranks data has been saved to /Users/ardadinc/Desktop/Market-Insight/Web_Parse_Data/Parsed_Data/Tipranks_Split_Data.csv


<h3>-----------------------------HedgeFollow.com Parsing Code-----------------------------<h3>
<h4>Code Will Take about a Minute<h4>
<h6>url = 'https://www.tipranks.com/calendars/stock-splits/upcoming'<h6>


In [4]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from datetime import datetime

# Function to convert date format from YYYY-MM-DD to MM/DD/YYYY
def convert_date_format(date_str):
    if pd.isna(date_str) or date_str == "N/A":
        return "N/A"
    return datetime.strptime(date_str, '%Y-%m-%d').strftime('%m/%d/%Y')

# Setup Chrome options
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Run in headless mode
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

# Setup the Chrome driver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# URL of the webpage to scrape
url = "https://hedgefollow.com/upcoming-stock-splits.php"

# Open the webpage
driver.get(url)

# Wait for the table to be present and visible
wait = WebDriverWait(driver, 30)
table = wait.until(EC.presence_of_element_located((By.ID, "latest_splits")))

# Initialize lists to store the data
stocks = []
exchanges = []
companies = []
split_ratios = []
ex_dates = []
announcement_dates = []

# Iterate over the rows in the table
for row in table.find_elements(By.TAG_NAME, "tr")[1:]:  # Skip the header row
    cols = row.find_elements(By.TAG_NAME, "td")
    if len(cols) == 6:  # Adjust the number of columns to match your data
        stocks.append(cols[0].text.strip())
        exchanges.append(cols[1].text.strip())
        companies.append(cols[2].text.strip())
        split_ratios.append(cols[3].text.strip())
        ex_dates.append(cols[4].text.strip())
        announcement_dates.append(cols[5].text.strip())

# Create a DataFrame from the data
data = {
    'Stock': stocks,
    'Exchange': exchanges,
    'Company Name': companies,
    'Split Ratio': split_ratios,
    'Ex-Date': ex_dates,
    'Announcement Date': announcement_dates
}
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
csv_file = '/Users/ardadinc/Desktop/Market-Insight/Web_Parse_Data/Parsed_Data/HedgeFollows_Split_Data.csv'
df.to_csv(csv_file, index=False)

print(f"Data has been saved to {csv_file}")

# Close the browser
driver.quit()

# Filter the data

# Read the CSV file
df = pd.read_csv(csv_file)

# Filter rows for reverse splits (where b > a in the split ratio a:b)
reverse_splits = []
for index, row in df.iterrows():
    split_ratio = row['Split Ratio']
    a, b = map(int, split_ratio.split(':'))
    if b > a:
        reverse_splits.append(row)

# Create a new DataFrame with the filtered data
reverse_splits_df = pd.DataFrame(reverse_splits)

# Change the date format
reverse_splits_df['Ex-Date'] = reverse_splits_df['Ex-Date'].apply(convert_date_format)

# Rename 'Ex-Date' column to 'Date'
reverse_splits_df = reverse_splits_df.rename(columns={'Ex-Date': 'Date'})
# Rename 'Ex-Date' column to 'Date'
reverse_splits_df = reverse_splits_df.rename(columns={'Stock': 'Symbol'})

# Remove the columns 'Announcement Date' and 'Exchange'
reverse_splits_df = reverse_splits_df.drop(columns=['Announcement Date', 'Exchange'])

# Reorder the columns to move 'Date' to be the first column
reverse_splits_df = reverse_splits_df[['Date', 'Symbol', 'Company Name', 'Split Ratio']]

# Save the filtered data to a new CSV file
output_file_path = '/Users/ardadinc/Desktop/Market-Insight/Web_Parse_Data/Parsed_Data/HedgeFollows_Split_Data.csv'
reverse_splits_df.to_csv(output_file_path, index=False)

print(f"Filtered data has been saved to {output_file_path}")


Data has been saved to /Users/ardadinc/Desktop/Market-Insight/Web_Parse_Data/Parsed_Data/HedgeFollows_Split_Data.csv
Filtered data has been saved to /Users/ardadinc/Desktop/Market-Insight/Web_Parse_Data/Parsed_Data/HedgeFollows_Split_Data.csv


<h2>-----------------------------Combined Parsed Code-----------------------------<h2>



In [5]:
import pandas as pd
from datetime import datetime

# Function to read and combine data from multiple CSV files
def read_and_combine_csv_files(file_paths):
    combined_df = pd.DataFrame()
    
    for file_path in file_paths:
        df = pd.read_csv(file_path)
        combined_df = pd.concat([combined_df, df], ignore_index=True)
    
    return combined_df

# Function to convert date format from MM/DD/YYYY to datetime object
def convert_to_datetime(date_str):
    try:
        return datetime.strptime(date_str, '%m/%d/%Y')
    except ValueError:
        return None

# Function to choose the most recent date for each unique ticker
def get_most_recent_dates(df):
    df['Date'] = df['Date'].apply(convert_to_datetime)
    most_recent_df = df.loc[df.groupby('Symbol')['Date'].idxmax()]
    return most_recent_df

# List of input CSV files
file_paths = [
    '/Users/ardadinc/Desktop/Market-Insight/Web_Parse_Data/Parsed_Data/HedgeFollows_Split_Data.csv',
    '/Users/ardadinc/Desktop/Market-Insight/Web_Parse_Data/Parsed_Data/StockAnalysis_Split_Data.csv',
    '/Users/ardadinc/Desktop/Market-Insight/Web_Parse_Data/Parsed_Data/Tipranks_Split_Data.csv',
    '/Users/ardadinc/Desktop/Market-Insight/Web_Parse_Data/Parsed_Data/API_Parsed_Data/stock_splits_polygon.csv'
]

# Combine data from multiple CSV files
combined_df = read_and_combine_csv_files(file_paths)

# Get the most recent dates for each unique ticker
result_df = get_most_recent_dates(combined_df)

# Sort by Date in descending order
result_df = result_df.sort_values(by='Date', ascending=False)

# Convert the Date column back to MM/DD/YYYY format
result_df['Date'] = result_df['Date'].dt.strftime('%m/%d/%Y')

# Save the result to a new CSV file
output_file_path = '/Users/ardadinc/Desktop/Market-Insight/Web_Parse_Data/Parsed_Data/Combined_Split_Data.csv'
result_df.to_csv(output_file_path, index=False)

print(f"Combined data has been saved to {output_file_path}")


Combined data has been saved to /Users/ardadinc/Desktop/Market-Insight/Web_Parse_Data/Parsed_Data/Combined_Split_Data.csv


<h2>-----------------------------Send Code To PRIA-----------------------------<h2>

In [6]:
import pandas as pd
from datetime import datetime, timedelta

# Function to filter data from the past week and future dates
def filter_past_week_and_future_data(df, date_column):
    # Get today's date
    today = datetime.today()
    # Calculate the date one week ago
    one_week_ago = today - timedelta(days=7)
    
    # Convert the date column to datetime format
    df[date_column] = pd.to_datetime(df[date_column], format='%m/%d/%Y')
    
    # Filter the DataFrame for rows within the past week and future dates
    filtered_data = df[df[date_column] >= one_week_ago]
    return filtered_data

# Path to the input CSV file (X.csv)
input_file_path = '/Users/ardadinc/Desktop/Market-Insight/Web_Parse_Data/Parsed_Data/Combined_Split_Data.csv'
# Path to the output CSV file (Y.csv)
output_file_path = '/Users/ardadinc/Desktop/Market-Insight/Web_Parse_Data/Alpha PRIA/TestData.csv'

# Read the input CSV file
df = pd.read_csv(input_file_path)

# Filter data from the past week and future dates
filtered_df = filter_past_week_and_future_data(df, 'Date')

# Format the Date column back to MM/DD/YYYY
filtered_df['Date'] = filtered_df['Date'].dt.strftime('%m/%d/%Y')

# Save the filtered data to the output CSV file
filtered_df.to_csv(output_file_path, index=False)

print(f"Filtered data has been saved to {output_file_path}")


Filtered data has been saved to /Users/ardadinc/Desktop/Market-Insight/Web_Parse_Data/Alpha PRIA/TestData.csv


/var/folders/r6/0smcfgt9415gwpc747f7ghth0000gn/T/ipykernel_64025/3856053443.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Date'] = filtered_df['Date'].dt.strftime('%m/%d/%Y')
